In [1]:
import pandas as pd
import numpy as np
import pylab as pl
import lxml
from bs4 import BeautifulSoup
import urllib2
import re

% pylab inline

Populating the interactive namespace from numpy and matplotlib


# Get Web Scraper Working Once

In [2]:

year = str("2014")
school = str("K765")


# Read in html and turn into soup
html = urllib2.urlopen("http://schools.nyc.gov/AboutUs/funding/schoolbudgets/GalaxyAllocationFY" + year + ".htm?BSSS_INPUT=" + school).read()
soup = BeautifulSoup(html, 'html.parser')

# Find the table and Total Budget for the year from there
try:
    table = soup.find('table', {'class': 'vgalaxy'})
    table.find('td', {'align': 'left'})
    th = table.find('td', text='Total')
    td = th.findNext('td')
    total = td.text
    
# If not available input NaN
except AttributeError: 
    total = np.nan

print(total)

nan


# Turn Webscraper into time period scraper and attach to Dataframe

In [3]:

years = np.arange(2006, 2018)
cols = ["dbn"]
for i in years:  
    cols.append(i)

budgetsdf = pd.DataFrame(columns=cols)
budget = []

budget.append(school)

for yr in years:
    
    try:
        # Read in html and turn into soup
        html = urllib2.urlopen("http://schools.nyc.gov/AboutUs/funding/schoolbudgets/GalaxyAllocationFY" + str(yr) + ".htm?BSSS_INPUT=" + school).read()
        soup = BeautifulSoup(html, 'html.parser')

        # Find the table and Total Budget for the year from there
        table = soup.find('table', {'class': 'vgalaxy'})
        table.find('td', {'align': 'left'})
        th = table.find('td', text='Total')
        td = th.findNext('td')
        total = td.text


    # If not available input NaN
    #except urllib2.HTTPError, AttributeError:
    except:
        total = np.nan

    budget.append(total)
    
print(budget)
budgetsdf.loc[len(budgetsdf)] = budget
budgetsdf.head()

['K765', nan, nan, nan, nan, nan, nan, nan, nan, nan, u'1,243,907', u'1,920,869', u'2,428,573']


,dbn,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,K765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"1,243,907","1,920,869","2,428,573"


# Read in list of all Schools to create for loop to get all budgets

In [4]:
# REad in school class size data
ClassSize2018 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/32A4D266-83DB-439D-AE73-EDD1BC23E9AD/0/February2018_Avg_ClassSize_School.xlsx",header=7)
ClassSize2017 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/11A3268A-AA1B-4475-BA79-A9D52883DC08/0/February2017_Avg_ClassSize_School.xlsx",header=7)
ClassSize2016 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/3B90A8E1-5B32-4B86-9BB3-4FCFC30A8182/0/February2016_Avg_ClassSize_School.xlsx",header=6)
ClassSize2015 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/5EC03778-74AD-45FF-9951-44441AE570F2/0/SchoolLevelDetailSummary_Updated_2015.xlsx",header=10)
ClassSize2014 = pd.read_excel("http://schools.nyc.gov/offices/d_chanc_oper/budget/dbor/DBOR_CLASS_SIZE/FY14_Data/Updated_School_level_Detail_Summary_2014_02_13.xlsx",header=10)
ClassSize2013 = pd.read_excel("http://schools.nyc.gov/offices/d_chanc_oper/budget/dbor/DBOR_CLASS_SIZE/FY13_Data/20122013UpdatedSchoolLevelDetailFinal%202012_12_14.xlsx",header=10)
ClassSize2012 = pd.read_excel("http://schools.nyc.gov/offices/d_chanc_oper/budget/dbor/DBOR_CLASS_SIZE/FY12_Data/Updated_School_Level_Detail_21512.xls",header=10)
ClassSize2011 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/5E4CE3DD-739E-4DEF-86CF-AEBFD87715BB/0/20102011PreliminarySchoolLevelDetail.xls",header=10)
ClassSize2010 = pd.read_excel("http://schools.nyc.gov/NR/rdonlyres/B887E47A-30EB-4D80-A5DA-46946B67C239/0/20102011UpdatedSchoolLevelDetail.xls",header=10)
# ClassSize2009 = pd.read_excel("")
# ClassSize2008 = pd.read_excel("")
# ClassSize2007 = pd.read_excel("")
# ClassSize2006 = pd.read_excel("")


In [5]:
# Create arrays of unique schools per year and combine
SchoolsList = list(ClassSize2018.DBN.str[2:].unique())
SchoolsList.extend(list(ClassSize2017.DBN.str[2:].unique()))
SchoolsList.extend(list(ClassSize2016.DBN.str[2:].unique()))
SchoolsList.extend(list(ClassSize2015["SCHOOL CODE"].unique()))
SchoolsList.extend(list(ClassSize2014["SCHOOL CODE"].unique()))
SchoolsList.extend(list(ClassSize2013["SCHOOL CODE"].unique()))
SchoolsList.extend(list(ClassSize2012["SCHOOL CODE"].unique()))
SchoolsList.extend(list(ClassSize2011["SCHOOL CODE"].unique()))
SchoolsList.extend(list(ClassSize2010["SCHOOL CODE"].unique()))

# Determine unique values and print count
SchoolsList = np.unique(SchoolsList)

len(SchoolsList)

1645

Webscraper seemed to not be able to do complete list in one go, so decided to break down by Borough to split it up and do each one separately and combine later.

In [6]:
# Create Borough specific Schools lists
# Brooklyn
r = re.compile(".*K")
SchoolsListK = filter(r.match, SchoolsList)

# Manhattan
r = re.compile(".*M")
SchoolsListM = filter(r.match, SchoolsList)

# Queens
r = re.compile(".*Q")
SchoolsListQ = filter(r.match, SchoolsList)

# Bronx
r = re.compile(".*X")
SchoolsListX = filter(r.match, SchoolsList)

# Staten Island
r = re.compile(".*R")
SchoolsListR = filter(r.match, SchoolsList)

### Change Schools List to each borough and then save to csv 

In [18]:
years = np.arange(2006, 2018)
cols = ["dbn"]
for i in years:  
    cols.append(i)
    
budgetsdf = pd.DataFrame(columns=cols)

for sc in SchoolsListK:
    print(sc)
    
    school = sc
    
    budget = []

    budget.append(school)

    for yr in years:

        try:
            # Read in html and turn into soup
            html = urllib2.urlopen("http://schools.nyc.gov/AboutUs/funding/schoolbudgets/GalaxyAllocationFY" + str(yr) + ".htm?BSSS_INPUT=" + school).read()
            soup = BeautifulSoup(html, 'html.parser')

            # Find the table and Total Budget for the year from there
            table = soup.find('table', {'class': 'vgalaxy'})
            table.find('td', {'align': 'left'})
            th = table.find('td', text='Total')
            td = th.findNext('td')
            total = td.text


        # If not available input NaN
        #except urllib2.HTTPError, AttributeError:
        except:
            total = np.nan

        budget.append(total)

    budgetsdf.loc[len(budgetsdf)] = budget

budgetsdf.head()

K001
K002
K003
K005
K006
K007
K008
K009
K010
K011
K012
K013
K014
K015
K016
K017
K018
K019
K020
K021
K022
K023
K024
K025
K026
K027
K028
K029
K030
K031
K032
K034
K035
K038
K039
K040
K041
K044
K045
K046
K048
K050
K051
K052
K054
K056
K057
K058
K059
K061
K062
K065
K066
K067
K068
K069
K071
K072
K073
K075
K078
K081
K084
K086
K088
K089
K090
K091
K092
K093
K094
K095
K096
K097
K098
K099
K100
K101
K102
K103
K104
K105
K106
K107
K108
K109
K110
K112
K113
K114
K115
K116
K118
K119
K120
K121
K122
K123
K124
K126
K127
K128
K130
K131
K132
K133
K134
K135
K136
K137
K138
K139
K145
K146
K147
K149
K150
K151
K152
K153
K154
K155
K156
K157
K158
K159
K160
K161
K162
K163
K164
K165
K166
K167
K168
K169
K170
K171
K172
K174
K176
K177
K178
K179
K180
K181
K184
K185
K186
K187
K188
K189
K190
K191
K192
K193
K194
K195
K196
K197
K198
K199
K200
K201
K202
K203
K204
K205
K206
K207
K208
K209
K211
K212
K213
K214
K215
K216
K217
K218
K219
K220
K221
K222
K223
K224
K225
K226
K227
K228
K229
K230
K233
K234
K235
K236
K238
K239
K240
K241


,dbn,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,K001,NaN,NaN,NaN,NaN,NaN,"9,992,651","10,143,921","10,648,403","11,236,206","11,876,960","12,679,061","13,666,442"
1,K002,NaN,NaN,NaN,NaN,NaN,"4,450,728","4,490,175","4,307,041","4,230,547","4,118,972","4,182,057","4,634,833"
2,K003,NaN,NaN,NaN,NaN,NaN,"4,918,217","5,072,110","4,904,738","4,842,447","4,979,315","4,896,431","4,870,818"
3,K005,NaN,NaN,NaN,NaN,NaN,"4,793,360","4,336,877","4,096,694","3,791,380","3,993,529","3,460,115","3,591,103"
4,K006,NaN,NaN,NaN,NaN,NaN,"5,824,302","5,899,457","5,982,643","6,163,292","6,617,531","7,126,444","8,268,231"


In [19]:
budgetsdf.tail()

,dbn,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
497,K821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,K839,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
499,K889,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
500,K890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,K971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
budgetsdf.to_csv("school_total_budget_Brooklyn.csv")

In [21]:
# Not repeatable step just by running code
# Read in all budgets
budgetsBK = pd.read_csv('school_total_budget_Brooklyn.csv')
budgetsBX = pd.read_csv('school_total_budget_Bronx.csv')
budgetsMN = pd.read_csv('school_total_budget_Manhattan.csv')
budgetsQN = pd.read_csv('school_total_budget_Queens.csv')
budgetsSI = pd.read_csv('school_total_budget_Richmond.csv')

In [29]:
# Merge data sets
budgets = pd.concat([budgetsBK,budgetsBX,budgetsMN,budgetsQN,budgetsSI])

# Drop first couple of columns
budgets.drop(['2006', '2007', '2008', '2009', '2010', 'Unnamed: 0'], axis=1, inplace=True)

In [30]:
budgets.head()

,dbn,2011,2012,2013,2014,2015,2016,2017
0,K001,"9,992,651","10,143,921","10,648,403","11,236,206","11,876,960","12,679,061","13,666,442"
1,K002,"4,450,728","4,490,175","4,307,041","4,230,547","4,118,972","4,182,057","4,634,833"
2,K003,"4,918,217","5,072,110","4,904,738","4,842,447","4,979,315","4,896,431","4,870,818"
3,K005,"4,793,360","4,336,877","4,096,694","3,791,380","3,993,529","3,460,115","3,591,103"
4,K006,"5,824,302","5,899,457","5,982,643","6,163,292","6,617,531","7,126,444","8,268,231"


In [31]:
budgets.to_csv('school_total_budget.csv')


# Webscraping to get full table and pull out specific things (PTA funding?)

In [ ]:
tablecontents = table.find_all('td')

# Create blank dataframe
new_table = pd.DataFrame(columns=range(0,2), index = [0])

# Create for loop tp run through table
for row in range((len(tablecontents)/2) - 1):

    # Define Text and Value data points
    text1 = tablecontents[2+(row*2)].text
    value1 = tablecontents[3+(row*2)].text
    
    # Add to dataframe
    new_table.loc[row] = [text1, value1.replace(",","")]

new_table